In [ ]:
# Load data
test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e1/sample_submission.csv')

# Feature engineering
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['day_of_week'] = test['date'].dt.dayofweek
test['is_weekend'] = test['day_of_week'].isin([5, 6]).astype(int)

# Scale numerical features
test[numerical_features] = scaler.transform(test[numerical_features])

# Encode categorical features
encoded_test_features = pd.DataFrame(encoder.transform(test[categorical_features]), index=test.index)
encoded_test_features.columns = encoder.get_feature_names_out(categorical_features)

# Combine numerical and encoded categorical features
X_test = pd.concat([test[numerical_features], encoded_test_features], axis=1)

# Predict on test set
y_pred_test_log = model.predict(X_test)
y_pred_test = np.expm1(y_pred_test_log)

# Add predictions to test DataFrame
test['predicted_sales'] = y_pred_test

# Submission csv
sample_submission['num_sold'] = y_pred_test
sample_submission.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission file created: submission.csv")